## OpenClean-Re Examples

In [1]:
import os
os.chdir('../../')

from openclean.patternfinder import PatternFinder
import pandas as pd
import csv, time, gc

In [2]:
os.getcwd()

'/home/munaf/PycharmProjects/OpenRefine_Regex/src/openclean'

In [3]:
PATH = os.path.join('/home/munaf/PycharmProjects/OpenRefine_Regex/')
DATADIR = os.path.join(PATH, "data/open2test/")
OUTPUTDIR = os.path.join(PATH, "out/open2test/")

### Parameters

- <b>series:</b> input column. PatternFinder can injest:
    - a dict() in the form value:frequency
    - a list() of all row values

- <b>frac:</b> sample size in percentage

- <b>aligner:</b> Select an aligner
    - None - no aligner specified. This forces the patternfinder to use a lightweight version which clusters columns together on # of tokens identified
    - 'comb' - _default_ - the heavyweight aligner which iterates through all possible pairwise alignments, calculates the [TDE distance](https://vldb.org/pvldb/vol11/p1165-he.pdf), and clusters them using DBSCAN. Note: that all hyperparameters have been heuristically preset for now. A future version will make them configurable.     
    - ['lingpy'](http://lingpy.org/intro.html) - performs multiple sequence alignment on phonetic sounds. Prelim tests showed poor performance.

- <b>distance:</b> Distances available:
    - 'TDE' 
    - 'ETDE' (enhanced: TDE: also considers the supported domain datatypes when calculating distance. This is _default_ when running encode_and_find()) 
    - 'ABS' (Absolute - difference b/w the cardinality of rows

- (deprecated) <b>tokenizer:</b> Select a tokenizer
    - 'smart' - searches the master datasets and replaces found values with identified classes followed by tokenizing. Currently supported include:
        - GeoSpatial(Country, County, State),
        - Date(Weekday, Month),
        - Address(Street, SUD) and 
        - Business Entity(e.g. LLC, INC etc)
    - 'punc' - _default_ - identifies and splits tokens on punctuation 
      

- (deprecated) <b>encode:</b> set to true if you want to replace row values not identified in the master data with their internal representations (alpha, alphanum, numeric, punc) ignoring intermediate representations (e.g. myXXXXX) prior to alignment.

#### the deprecated parameters have been replaced with a dedicated endpoint

### Methods

#####  PatternFinder(series).find()
- tokenizes columns on punctuation
- aligns the tokens
- encodes them into the internal representations (doesnt support domain specific data types yet)
- calculates the regex
- detects outliers

##### PatternFinder(series, distance='ETDE').encode_and_find()
- tokenizes columns on punctuation
- encodes them into the internal representations (supports domain specific data types)
- aligns the internally represented tokens
- calculates the regex
- detects outliers

##### PatternFinder(series).evaluate(column, encode)
- tokenizes column in a similar way to the original series (set using the encode flag)
- encodes into representations deciding whether to encode
- matches the representations to the series patterns for each cluster - pattern and generalized
- returns a dict with pattern and % of the column that matched each pattern

_________________________________________________________________________________________________________________

### Pattern Finder Workflow
I've used some major components here. This can be modified depending on differing use cases. e.g. encode_and_find()

In [4]:
filename = 'location.csv'
FILEPATH = os.path.join(DATADIR, filename)

In [5]:
df = pd.read_csv(FILEPATH, sep='\t', squeeze=False, header=None)
if df.shape[1] == 2:
    series = df.set_index(0).squeeze().to_dict()
elif df.shape[0] == 1:
    series = df[0].tolist()
else:
    raise ValueError(file)

In [6]:
#column 0 -> row value, column 1 -> frequency (optional - for PatternFinder object)
# other preprocessing done includes lower(), replace('\'',''), sampling
series = df[0]
df.head()

,0,1
0,137 HEBERTON AVENUE STATEN ISLAND NY (40.63677...,1
1,261 LINCOLN AVENUE STATEN ISLAND NY (40.576986...,10
2,1950 CLOVE ROAD STATEN ISLAND NY (40.606004051...,1
3,20 COURT STREET STATEN ISLAND NY (40.627181943...,1
4,280 BEACH AVENUE STATEN ISLAND NY (40.60667400...,1


In [7]:
from openclean.tokenize.regex import RegexTokenizer

rt = RegexTokenizer().tokenize(series)
rt

0    [137,  , HEBERTON,  , AVENUE,  , STATEN,  , IS...
1    [261,  , LINCOLN,  , AVENUE,  , STATEN,  , ISL...
2    [1950,  , CLOVE,  , ROAD,  , STATEN,  , ISLAND...
3    [20,  , COURT,  , STREET,  , STATEN,  , ISLAND...
4    [280,  , BEACH,  , AVENUE,  , STATEN,  , ISLAN...
5    [162,  , ST,  , MARY, ', S,  , AVENUE,  , STAT...
6    [7,  , BENTON,  , COURT,  , STATEN,  , ISLAND,...
7    [330,  , BARD,  , AVENUE,  , STATEN,  , ISLAND...
8    [25,  , BAY,  , TERRACE,  , STATEN,  , ISLAND,...
9    [395,  , VICTORY,  , BOULEVARD,  , STATEN,  , ...
Name: 0, dtype: object

In [8]:
from openclean.align.combinatorics import CombAligner

ca = CombAligner().get_aligned(rt)
ca

,tokens,cluster,aligned
0,"[137, , HEBERTON, , AVENUE, , STATEN, , IS...",0,"[137, , HEBERTON, , AVENUE, , STATEN, , IS..."
1,"[261, , LINCOLN, , AVENUE, , STATEN, , ISL...",0,"[261, , LINCOLN, , AVENUE, , STATEN, , ISL..."
2,"[1950, , CLOVE, , ROAD, , STATEN, , ISLAND...",0,"[1950, , CLOVE, , ROAD, , STATEN, , ISLAND..."
3,"[20, , COURT, , STREET, , STATEN, , ISLAND...",0,"[20, , COURT, , STREET, , STATEN, , ISLAND..."
4,"[280, , BEACH, , AVENUE, , STATEN, , ISLAN...",0,"[280, , BEACH, , AVENUE, , STATEN, , ISLAN..."
5,"[162, , ST, , MARY, ', S, , AVENUE, , STAT...",-1,"[162, , ST, , MARY, ', S, , AVENUE, , STAT..."
6,"[7, , BENTON, , COURT, , STATEN, , ISLAND,...",0,"[7, , BENTON, , COURT, , STATEN, , ISLAND,..."
7,"[330, , BARD, , AVENUE, , STATEN, , ISLAND...",0,"[330, , BARD, , AVENUE, , STATEN, , ISLAND..."
8,"[25, , BAY, , TERRACE, , STATEN, , ISLAND,...",0,"[25, , BAY, , TERRACE, , STATEN, , ISLAND,..."
9,"[395, , VICTORY, , BOULEVARD, , STATEN, , ...",0,"[395, , VICTORY, , BOULEVARD, , STATEN, , ..."


In [9]:
# remove alignment outliers
outliers = ca[ca['cluster']==-1]
non_outliers = ca[ca['cluster']!=-1]

# create frequency dict from the new aligned column 
ca['column'] = non_outliers.tokens.str.join('')
full_ca = ca.merge(df,left_on='column',right_on=0)
full_ca['aligned_hashable'] = full_ca.aligned.str.join('')
frequency_dict = full_ca[['aligned_hashable',1]].set_index('aligned_hashable').to_dict()[1]

In [10]:
from openclean.regex.compiler import RegexCompiler

# encode the aligned tokens to internal representations of RegexRows and RegexTokens
# note: this does not use master data classes

regex_matrix, _ = RegexCompiler.compile(aligned_rows=non_outliers['aligned'].to_numpy(), frequency_dict=frequency_dict)
regex_matrix[1]

RegexRow([RegexToken('NUMERIC',3,'261', 10), RegexToken('PUNC',1,' ', 10), RegexToken('ALPHA',7,'LINCOLN', 10), RegexToken('PUNC',1,' ', 10), RegexToken('ALPHA',6,'AVENUE', 10), RegexToken('PUNC',1,' ', 10), RegexToken('ALPHA',6,'STATEN', 10), RegexToken('PUNC',1,' ', 10), RegexToken('ALPHA',6,'ISLAND', 10), RegexToken('PUNC',1,' ', 10), RegexToken('ALPHA',2,'NY', 10), RegexToken('PUNC',1,' ', 10), RegexToken('PUNC',1,'(', 10), RegexToken('NUMERIC',2,'40', 10), RegexToken('PUNC',1,'.', 10), RegexToken('NUMERIC',12,'576986694649', 10), RegexToken('PUNC',1,' ', 10), RegexToken('PUNC',1,'-', 10), RegexToken('NUMERIC',2,'74', 10), RegexToken('PUNC',1,'.', 10), RegexToken('NUMERIC',12,'104670051676', 10), RegexToken('PUNC',1,')', 10)], 10)

In [11]:
# create regex string
# RegexCompiler.generate_regex_from_matrix(regex_matrix)

# or manually
from openclean.regex.pattern_generator import RegexTokensPattern

full_regex = general_full_regex = str()
matrix = regex_matrix

for col in range(matrix[0].get_size()):
    column_regex = RegexTokensPattern()
    for row in range(len(matrix)):
        token = matrix[row].get_regex_token(col).get_token()
        
        # inserts token incrementally to build the piecewise regex
        column_regex.insert_token(token) # can be used to add other 
        # rows to update the pattern e.g. those identified as outliers
    
    fr, gfr = column_regex.condense()
    full_regex += fr + ' '
    general_full_regex += gfr + ' '
    
print(full_regex)

[NUMERIC(1-4)] [PUNC(\S)] [ALPHA(3-8)] [PUNC(\S)] [ALPHA(4-9) | STREET(6-6)] [PUNC(\S)] [ALPHA(6-6)] [PUNC(\S)] [ALPHA(6-6)] [PUNC(\S)] [ALPHA(2-2)] [PUNC(\S)] [PUNC(()] [NUMERIC(2-2)] [PUNC(.)] [NUMERIC(9-12)] [PUNC(\S)] [PUNC(-)] [NUMERIC(2-2)] [PUNC(.)] [NUMERIC(10-12)] [PUNC())] 


________________________________________________________________________________________

### Examples

In [12]:
filename = 'email.csv'
FILEPATH = os.path.join(DATADIR, filename)

In [13]:
df = pd.read_csv(FILEPATH, sep='\t', squeeze=False, header=None)
if df.shape[1] == 2:
    series = df.set_index(0).squeeze().to_dict()
elif df.shape[0] == 1:
    series = df[0].tolist()
else:
    raise ValueError(file)

In [14]:
df.head(15)

,0,1
0,BBASSEN@EWHOWELL.COM,2
1,EDWARDHICKSRA@YAHOO.COM,1
2,JB@MESZENGINEERING.COM,1
3,JEMMCO9@GMAIL.COM,1
4,DWARFIELD@BANCKER.COM,1
5,MIKIGAL@AOL.COM,2
6,SHLOMONKUB@GMAIL.COM,1
7,GLENBAKHSHI@AOL.COM,1
8,FVILLANO@FPVCONSULTING.COM,1
9,DENNISAUGUST@ICLOUD.COM,1


### Light weight: tokenizer:punc, aligner:None

In [15]:
pf = PatternFinder(series=series,
        tokenizer='punc',
        aligner = None,
        frac=.01)

start_time = time.time()
   
regex = pf.find()

end_time = time.time()

print('time taken: {}'.format(end_time - start_time))

time taken: 0.061486244201660156


In [16]:
pf.regex

,index,Pattern,Generalized,Proportion,Sample
0,5,[ALPHA(1-22) | ALPHANUM(4-18)] [PUNC(@-.)] [AL...,[ALPHANUM(1-22)] [PUNC(@-.)] [ALPHA(3-23)] [PU...,0.858696,"[bestinteriorsinc, @, yahoo, ., com]"
1,3,[joycon1st(9-9) | ALPHA(6-17)] [PUNC(.@)] [ALP...,[ALPHANUM(6-17)] [PUNC(.@)] [ALPHA(3-8)],0.018116,"[joycon1st, ., com]"
2,7,[ALPHA(1-8) | frecchia51(10-10)] [PUNC(.@-_)] ...,[ALPHA(1-10)] [PUNC(.@-_)] [ALPHANUM(2-12)] [P...,0.105072,"[arcadius, ., office, @, gmail, ., com]"
3,9,[ALPHA(1-6)] [PUNC(.-@)] [ALPHA(1-10)] [PUNC(@...,[ALPHA(1-6)] [PUNC(.-@)] [ALPHA(1-10)] [PUNC(@...,0.018116,"[lana, ., harrinanan, @, trinity, -, solar, .,..."


In [17]:
pf.regex.Generalized.values
# reasonable results but punctuations aggregated and it's difficult to tell this is an email column

array(['[ALPHANUM(1-22)] [PUNC(@-.)] [ALPHA(3-23)] [PUNC(.)] [ALPHA(3-3)] ',
       '[ALPHANUM(6-17)] [PUNC(.@)] [ALPHA(3-8)] ',
       '[ALPHA(1-10)] [PUNC(.@-_)] [ALPHANUM(2-12)] [PUNC(@-.)] [ALPHA(2-17)] [PUNC(.)] [ALPHA(3-3)] ',
       '[ALPHA(1-6)] [PUNC(.-@)] [ALPHA(1-10)] [PUNC(@-.)] [ALPHA(1-7)] [PUNC(-@.)] [ALPHA(2-7)] [PUNC(.)] [ALPHA(3-3)] '],
      dtype=object)

In [18]:
pf.outliers

{'alson.electrical.live@gmail.com': 1,
 'arc-ed@nyc.rr.com': 1,
 'crsafetygroup.com': 1,
 'daveyaimo@gmail': 1,
 'g.intra-agi@hotmail.com': 1,
 'joycon1st.com': 1,
 'lana.harrinanan@trinity-solar.com': 1,
 'matina@epochele': 1,
 'tadformfactorynyc.com': 1,
 'wilson@s-c-arch.com': 1}

______________________________________________________________________________________

### Middle weight: tokenizer:punc, aligner:COMB

In [19]:
pf = PatternFinder(series=series,
        tokenizer='punc',
        aligner = 'COMB',
        frac=.01)

start_time = time.time()
   
regex = pf.find()

end_time = time.time()

print('time taken: {}'.format(end_time - start_time))


time taken: 22.807870388031006


In [20]:
pf.regex

,index,Pattern,Generalized,Proportion,Sample
0,0,[ALPHA(2-22) | ALPHANUM(4-18)] [PUNC(@)] [ALPH...,[ALPHANUM(2-22)] [PUNC(@)] [ALPHA(3-23)] [PUNC...,0.847826,"[bestinteriorsinc, @, yahoo, ., com]"
1,1,[ALPHA(6-8)] [PUNC(.)] [ALPHA(3-8)] [PUNC(@)] ...,[ALPHA(6-8)] [PUNC(.)] [ALPHA(3-8)] [PUNC(@)] ...,0.018116,"[arcadius, ., office, @, gmail, ., com]"
2,-1,NaN,NaN,0.134058,NaN


In [21]:
pf.regex.Generalized.values
# same results as before without encoding. Just demonstrating this. Encoding will be used to aggregate similar token types together later on

array(['[ALPHANUM(2-22)] [PUNC(@)] [ALPHA(3-23)] [PUNC(.)] [ALPHA(3-3)] ',
       '[ALPHA(6-8)] [PUNC(.)] [ALPHA(3-8)] [PUNC(@)] [ALPHA(4-9)] [PUNC(.)] [ALPHA(3-3)] ',
       nan], dtype=object)

In [22]:
pf.outliers

{'alson.electrical.live@gmail.com': 1,
 'andrew@dynami-inc.com': 1,
 'arc-ed@nyc.rr.com': 1,
 'arnold@cd-ae.com': 2,
 'asf@only-if.com': 3,
 'crsafetygroup.com': 1,
 'daveyaimo@gmail': 1,
 'dluczak@si.rr.com': 1,
 'ed.fasttrack@gmail.com': 1,
 'frecchia51.fr@gmail.com': 1,
 'g.intra-agi@hotmail.com': 1,
 'info@ecc-ny.com': 1,
 'ip_statebuilder@yahoo.com': 1,
 'j@jockdeboer.com': 1,
 'jan.allen@amirit.com': 1,
 'jim.agreseta@jacarpentry.com': 1,
 'joe.dalonzo@cowbaycontracting.com': 2,
 'joycon1st.com': 1,
 'lana.harrinanan@trinity-solar.com': 1,
 'mason-aid0013@optonline.net': 1,
 'matina@epochele': 1,
 'mcmsafetynet.gmail.com': 1,
 'nelson@cb-construction.com': 1,
 'p.singh09@hotmail.com': 1,
 'patrick.mcalarney@lendlease.com': 1,
 'pete.gerolimatos@ifathomcs.com': 2,
 'rlepre@abra-avante.com': 1,
 'sze@360-gc.com': 2,
 'tadformfactorynyc.com': 1,
 'touchstone-builders.com': 1,
 'wilson@s-c-arch.com': 1}

_______________________________________________________________________________________

### Heavyweight: encode using masterdata
This approach aligns the internal representation of the tokens, instead of the tokens themselves and the distance (ETDE) is calculated between all supported class types

In [23]:
filename = 'location.csv'
FILEPATH = os.path.join(DATADIR, filename)

In [24]:
df = pd.read_csv(FILEPATH, sep='\t', squeeze=False, header=None)
if df.shape[1] == 2:
    series = df.set_index(0).squeeze().to_dict()
elif df.shape[0] == 1:
    series = df[0].tolist()
else:
    raise ValueError(file)

In [25]:
df.iloc[0].values

array(['137 HEBERTON AVENUE STATEN ISLAND NY (40.636779419508 -74.131313349744)',
       1], dtype=object)

In [26]:
pf = PatternFinder(series=series,
        tokenizer='smart',
        aligner='COMB', 
        distance='ETDE',
        frac=1)

start_time = time.time()
   
regex = pf.encode_and_find()

end_time = time.time()

print('time taken: {}'.format(end_time - start_time))

time taken: 0.4212801456451416


In [27]:
pf.regex

,index,Pattern,Generalized,Proportion,Sample
0,0,[NUMERIC(1-4)] [PUNC(\S)] [bay(3-3) | ALPHA(4-...,[NUMERIC(1-4)] [PUNC(\S)] [ALPHA(3-8)] [PUNC(\...,0.916667,25 bay terrace staten island ny (40.6066740001...
1,-1,NaN,NaN,0.083333,NaN


In [28]:
pf.regex.Generalized.iloc[0]

'[NUMERIC(1-4)] [PUNC(\\S)] [ALPHA(3-8)] [PUNC(\\S)] [STREET(4-9)] [PUNC(\\S)] [ALPHA(6-6)] [PUNC(\\S)] [STREET(6-6)] [PUNC(\\S)] [ALPHA(2-2)] [PUNC(\\S)] [PUNC(()] [NUMERIC(2-2)] [PUNC(.)] [NUMERIC(9-12)] [PUNC(\\S)] [PUNC(-)] [NUMERIC(2-2)] [PUNC(.)] [NUMERIC(10-12)] [PUNC())] '

In [29]:
pf.outliers
# all other row values have 22 tokens except 162 st marys avenue - it has 24 (st+<space>)

{'162 st marys avenue staten island ny (40.614004218866 -74.072557751113)': 2}

________________________________________________________________________________________________________________

### Pattern Finder Evaluate

In [30]:
# as a sanity check, applying the calculated patterns back to the same columns
pf = PatternFinder(series=series,
        tokenizer='smart',
        aligner='COMB',
        distance='ETDE',
        frac=1)

start_time = time.time()
   
regex = pf.encode_and_find()

end_time = time.time()

print('time taken: {}'.format(end_time - start_time))

time taken: 0.31163597106933594


In [31]:
pf.regex

,index,Pattern,Generalized,Proportion,Sample
0,0,[NUMERIC(1-4)] [PUNC(\S)] [bay(3-3) | ALPHA(4-...,[NUMERIC(1-4)] [PUNC(\S)] [ALPHA(3-8)] [PUNC(\...,0.916667,25 bay terrace staten island ny (40.6066740001...
1,-1,NaN,NaN,0.083333,NaN


In [32]:
# if encode_and_find used to generate the patterns, remember to set encode=True in evaluate (to use master data)
# some patterns with domain types were generalized to alpha/alphanum etc hence the 70% match for generalized 
pf.evaluate(series, encode=True)

{(0, 'Pattern'): 0.9166666666666666, (0, 'Generalized'): 0.7083333333333334}

________________________________________________________________________________________________________________

In [33]:
# without master data
pf = PatternFinder(series=series,
        tokenizer='punc',
        frac=1)

start_time = time.time()
   
regex = pf.find()

end_time = time.time()

print('time taken: {}'.format(end_time - start_time))

time taken: 0.27971577644348145


In [34]:
pf.evaluate(series)

{(0, 'Pattern'): 0.9166666666666666, (0, 'Generalized'): 0.9166666666666666}

________________________________________________________________________________________________________________

In [35]:
# or just a random junky column
new_col = [list(range(22)), list(range(22)), list(range(22))]
pf.evaluate(new_col)

{(0, 'Pattern'): 0.0, (0, 'Generalized'): 0.0}

________________________________________________________________________________________